In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"3,775,154","+8,206","182,259",+752,"1,732,460","+4,609","1,860,435","18,906",,,,,,North America,\n,,
1,,\nSouth America\n,"2,733,305","+1,129","100,406",+61,"1,813,243","+33,600","819,656","13,629",,,,,,South America,\n,,
2,,\nAsia\n,"2,799,182","+7,906","66,752",+122,"1,925,317","+4,843","807,113","19,769",,,,,,Asia,\n,,
3,,\nEurope\n,"2,533,366","+1,005","195,507",+23,"1,480,467","+1,104","857,392","5,401",,,,,,Europe,\n,,
4,,\nAfrica\n,"545,192",+129,"12,502",+2,"265,794",+11,"266,896",952,,,,,,Africa,\n,,


In [6]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"3,220,500",+501,"135,828",+6,"1,426,483",+55,"1,658,189","15,645","9,728",410,"40,163,765","121,322","331,049,917",North America
1,Brazil,"1,759,103",,"69,254",,"1,185,596","+33,129","504,253","8,318","8,274",326,"4,514,329","21,234","212,595,276",South America
2,India,"795,605",+763,"21,632",+9,"496,048",+88,"277,925","8,944",576,16,"11,024,491","7,987","1,380,307,868",Asia
3,Russia,"707,301",,"10,843",,"481,316",,"215,142","2,300","4,847",74,"22,388,195","153,411","145,936,153",Europe
4,Peru,"316,448",,"11,314",,"207,802",,"97,332","1,292","9,595",343,"1,863,278","56,495","32,981,172",South America


In [7]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,049,917","3,220,500",+501,"135,828",+6,"1,426,483",+55,"1,658,189","15,645","9,728",410,"40,163,765","121,322"
1,Brazil,South America,"212,595,276","1,759,103",,"69,254",,"1,185,596","+33,129","504,253","8,318","8,274",326,"4,514,329","21,234"
2,India,Asia,"1,380,307,868","795,605",+763,"21,632",+9,"496,048",+88,"277,925","8,944",576,16,"11,024,491","7,987"
3,Russia,Europe,"145,936,153","707,301",,"10,843",,"481,316",,"215,142","2,300","4,847",74,"22,388,195","153,411"
4,Peru,South America,"32,981,172","316,448",,"11,314",,"207,802",,"97,332","1,292","9,595",343,"1,863,278","56,495"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['French Guiana', 'CAR', 'Mayotte', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Guadeloupe', 'Gibraltar',
       'Brunei ', 'Aruba', 'Sint Maarten', 'Turks and Caicos',
       'French Polynesia', 'Macao', 'Saint Martin',
       'St. Vincent Grenadines', 'Curaçao', 'New Caledonia',
       'Falkland Islands', 'Montserrat', 'MS Zaandam'], dtype=object)

In [10]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331049917,3220500,501,135828,6,1426483,55,1658189,15645,9728,410,40163765,121322,Americas
1,Brazil,South America,212595276,1759103,NaN,69254,NaN,1185596,33129,504253,8318,8274,326,4514329,21234,Americas
2,India,Asia,1380307868,795605,763,21632,9,496048,88,277925,8944,576,16,11024491,7987,South-EastAsia
3,Russia,Europe,145936153,707301,NaN,10843,NaN,481316,NaN,215142,2300,4847,74,22388195,153411,Europe
4,Peru,South America,32981172,316448,NaN,11314,NaN,207802,NaN,97332,1292,9595,343,1863278,56495,Americas


In [11]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)